In [2]:
import os

root = os.getenv('PROJ_ROOT')
notebook_path = os.path.join(root, 'tiktok_niche/har_processing.ipynb')

file_name = os.path.basename(notebook_path)
print(file_name)

har_processing.ipynb


In [7]:
import json
import re
from haralyzer import HarParser
import pandas as pd
from datetime import datetime

# Load the HAR file
with open('content_creator_tips_search_Archive [24-10-16 11-01-58].har', 'r', encoding='utf-8') as f:
    har_parser = HarParser(json.loads(f.read()))

#data = har_parser.har_data
#print(type(data))

# Regex patterns for matching variables
patterns = {
    'username': re.compile(r'"uniqueId"\s*:\s*"([^"]+)"'),
    'nickname': re.compile(r'"nickname"\s*:\s*"([^"]+)"'),
    'bio': re.compile(r'"signature"\s*:\s*"([^"]+)"'),
    'follower_count': re.compile(r'"followerCount"\s*:\s*(\d+)'),
    #'hashtags': re.compile(r'"hashtagName"\s*:\s*"([^"]+)"'),
    'video_description': re.compile(r'"desc"\s*:\s*"([^"]+)"'),
    'upload_time': re.compile(r'"createTime"\s*:\s*(\d+)'),
    'duration': re.compile(r'"duration"\s*:\s*(\d+)'),
    'sound': re.compile(r'"title"\s*:\s*"([^"]+)"'),
    'view_count': re.compile(r'"playCount"\s*:\s*(\d+)'),
    'comment_count': re.compile(r'"commentCount"\s*:\s*(\d+)'),
    'like_count': re.compile(r'"diggCount"\s*:\s*(\d+)'),
    'save_count': re.compile(r'"collectCount"\s*:\s*(\d+)'),
    'share_count': re.compile(r'"shareCount"\s*:\s*(\d+)')
}


In [9]:
data = []

# Iterate over pages and entries in the HAR file
for page in har_parser.pages:
    for entry in page.entries:
        # Ensure that the entry has a 'text' field in its response content
        response_text = entry['response']['content'].get('text', '')
        if response_text:
            # Extract the data using regex
            extracted_data = {}
            for key, pattern in patterns.items():
                match = pattern.search(response_text)
                extracted_data[key] = match.group(1) if match else None

            # Convert UNIX time for upload_date if available
            #if extracted_data.get('upload_time'):
            #    extracted_data['upload_time'] = convert_unix_time(int(extracted_data['upload_time']))

            # Add the row to the dataset if relevant data was found
            if any(extracted_data.values()):
                data.append(extracted_data)


# Convert to a pandas DataFrame
df = pd.DataFrame(data)

# Show the first few rows of the DataFrame to verify the extraction
print(df.head())

        username                     nickname  \
0    coach.stone  Stone | Social Media Growth   
1    juliabroome         julez | social media   
2  theaaronchris                theaaronchris   
3      ugcravina  Ravina | Content Strategist   
4        evhandd                         Evan   

                                                 bio follower_count  \
0  Share your knowledge📲\nGrow \u0026 Make💵\nTikt...         289200   
1  social media manager / branding / creative str...         161600   
2  Cinematographer 🎥 • Photographer 📸  \n📍Mobile,...          11100   
3  Helping you grow \u0026 monetize on TikTok \nC...         102100   
4  📱Content Creator\n📚Content Tips \u0026 Tricks\...         188600   

          hashtags                                  video_description  \
0       coachstone  This is everything you need to become full-tim...   
1      contentplan  use THIS ‼️ to plan your content so that all o...   
2  contentcreators  How to upload high quality videos to Ti

In [10]:
df.to_csv('extracted_tiktok_data.csv', index=False)